# Writing regression tree from scratch
### Import libraries

In [1]:
import numpy as np
import pandas as pd
import time

### Create the node class
Each node contains attribute and threshold and their left and right child

In [2]:
class RegressionTreeNode(object):
    # Constructor
    def __init__(self, att, thr, left, right):  
        self.attribute = att
        self.threshold = thr
        # left and right are either binary classifications or references to 
        # decision tree nodes
        self.left = left     
        self.right = right   

    def print_tree(self,indent=''):
        # If prints the right subtree, corresponding to the condition x[attribute] > threshold
        # above the condition stored in the node
        if isinstance(self.right, np.float64):
            print(indent+'       ','pred=',self.right)
        else:
            self.right.print_tree(indent+'    ')
        
        print(indent,'if x['+str(self.attribute)+'] <=',self.threshold)
        
        if isinstance(self.left, np.float64):
            print(indent+'       ','pred=',self.left)
        else:
            self.left.print_tree(indent+'    ')

### Create the class for decision tree classifier
Algorithm ID3(x,y):
1. If termination condition applies return leaf with the mean value of y 
2. Determine all the thresholds 
2. Determine the best attribute and threshold (a,t) with the lowest MSE
3. Split the data based on the best attribute and threshold from 2
    a. Let (xl, yl) be the training examples for which x(a)<=t
    b. Let (xr, yr) be the training examples for which x(a)>t
4. Recursivley return decision tree node where attribute = a, threshold = t, leftchild = ID3(xl,yl), rightchild = ID3 (xr,yr)

In [3]:
class RegressionTreeClassifier(object):
    # constructor
    def __init__(self, max_depth=10, min_samples_split=10, accuracy=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.accuracy = accuracy
        
    def fit(self, data, labels):
        self.root = self._id3(data, labels, depth=0)
        
    def predict(self, test_data):
        pred = np.zeros(len(test_data), dtype=np.float64)
        for i in range(len(test_data)):
            pred[i] = self._predict(self.root, test_data[i])
        return pred
        
    def _id3(self, data, labels, depth):
        # check the base case (termination condition)
        mean_val = np.mean(labels)
        if depth==self.max_depth or len(data)<= self.min_samples_split or max(
                [mean_val, 1-mean_val])>= self.accuracy:
            return mean_val
        else:
            depth += 1
            all_thrs = self._get_all_thrs(data)
            # get the best attribute and threshold wth the highest gain
            best_split_col, best_split_val = self._get_best_split(data, labels, all_thrs)
            less, more = self._split_data(data, best_split_col, best_split_val)
            #recursivly build the tree
            left = self._id3(data[less], labels[less], depth)
            right = self._id3(data[more], labels[more], depth)

        return RegressionTreeNode(best_split_col, best_split_val, left, right)
    
    def _get_all_thrs(self, data):
        all_thrs = {}
        for index in range(data.shape[1]):
            all_thrs[index] = []
            unique_val = np.unique(data[:,index])
            
            for idx in range(len(unique_val)):
                if idx != 0:
                    current_val = unique_val[idx]
                    previous_val = unique_val[idx - 1]
                    thr = (current_val + previous_val)/2
                    all_thrs[index].append(thr)
        return all_thrs
        
    # Find the best cloumn to classify and the best threshold value
    def _get_best_split(self, data, labels, all_thrs):
        best_mse = 999
        for col_index in range(data.shape[1]):
            for thr in all_thrs[col_index]:
                less, more = self._split_data(data, col_index, thr)
                mse = self._calculate_mse(labels[less], labels[more])
                
                if mse < best_mse:
                    best_mse = mse
                    best_split_col = col_index
                    best_split_val = thr
        return best_split_col, best_split_val
    
    def _split_data(self, data, split_col, thr):
        less = data[:,split_col] <= thr
        more = data[:, split_col] > thr
        return less, more
    
    def _calculate_mse(self, y_below, y_above):
        left = np.mean(y_below)
        right = np.mean(y_above)
        mse = np.sum((y_below-left)**2) + np.sum((y_above-right)**2)
        return mse
   
    def _predict(self, dt_node, x):
        if isinstance(dt_node, np.float64):
            return dt_node
        if x[dt_node.attribute] <= dt_node.threshold:
            return self._predict(dt_node.left, x)
        else:
            return self._predict(dt_node.right, x)
     
    def display(self):
        print("model")
        self.root.print_tree()

### Laod and prepare data

In [4]:
x = []
y = []
infile = open("gamma_ray.txt","r")
for line in infile:
    y.append(int(line[-2:-1] =='g'))
    x.append(np.fromstring(line[:-2], dtype=float,sep=','))
infile.close()
    
x = np.array(x).astype(np.float32)
y = np.array(y) 

### Split data into trianing and testing set: 80% for training and 20% for testing

In [5]:
ind = np.random.permutation(len(y))
split_ind = int(len(y)*0.8)
x_train = x[ind[:split_ind]]
x_test = x[ind[split_ind:]]
y_train = y[ind[:split_ind]]
y_test = y[ind[split_ind:]]

### Take a toy dataset to run the program with minimum time

In [6]:
# Only 5000 data for training and 500 for testing
# Run this cell, only if you want to see the reuslt in quick otherwise skip this cell
x_train = x_train[:5000]
x_test = x_test[:500]
y_train = y_train[:5000]
y_test = y_test[:500]

### Fit / train the model

In [8]:
model = RegressionTreeClassifier(max_depth=5)
start = time.time()
model.fit(x_train, y_train)
elapsed_time = time.time()-start
print('Elapsed_time training  {0:.6f} '.format(elapsed_time)) 

Elapsed_time training  13.154929 


### Display the regression tree

In [9]:
model.display()

model
                        pred= 1.0
                 if x[2] <= 4.837650299072266
                        pred= 0.026476578411405296
             if x[8] <= 34.94029998779297
                        pred= 0.14285714285714285
                 if x[2] <= 2.44350004196167
                        pred= 1.0
         if x[9] <= 156.8125
                        pred= 0.07777777777777778
                 if x[2] <= 2.533249855041504
                        pred= 1.0
             if x[0] <= 63.6379508972168
                        pred= 0.5346534653465347
                 if x[1] <= 14.776599884033203
                        pred= 0.0784313725490196
     if x[0] <= 33.39979934692383
                        pred= 0.39919354838709675
                 if x[9] <= 171.15174865722656
                        pred= 0.6735751295336787
             if x[1] <= 10.003049850463867
                        pred= 0.10256410256410256
                 if x[2] <= 2.345900058746338
                        pred

### Training accuracy

In [10]:
train_pred = model.predict(x_train)
print('Mean square error test set:',np.mean(np.square(train_pred-y_train)))

Mean square error test set: 0.11459673767598433


### Testing accuracy

In [11]:
start = time.time()
test_pred = model.predict(x_test)
elapsed_time = time.time() - start
print('Elapsed_time testing {0:.6f}'.format(elapsed_time))
print('Mean square error test set:',np.mean(np.square(test_pred-y_test)))

Elapsed_time testing 0.004986
Mean square error test set: 0.14995741361048634


### Display 10 random regression results

In [13]:
idx = np.random.randint(0, 500, 10)
dict = {"x[0]": x_test[idx,0], "x[1]": x_test[idx,1], "x[2]": x_test[idx,2], "x[3]": x_test[idx,3], "x[4]": x_test[idx,4],
        "x[5]": x_test[idx,5], "x[6]": x_test[idx,6], "x[7]": x_test[idx,7], "x[8]": x_test[idx,8], "x[9]": x_test[idx,9],
        "labels": y_test[idx], "prediction": test_pred[idx]}
df = pd.DataFrame(dict)
pd.set_option('display.width', 1000)
print(df)

         x[0]       x[1]    x[2]    x[3]    x[4]        x[5]        x[6]       x[7]       x[8]        x[9]  labels  prediction
0   30.531099  12.690000  2.4014  0.4246  0.2361  -26.566900  -19.546101   9.588300   1.382300  243.460007       1    0.923077
1   20.698700  13.687200  2.4742  0.5671  0.3339   21.969400   16.062599  -3.845300  85.879303  161.453003       1    0.673575
2   55.164600  22.375000  3.2629  0.1850  0.0974   -7.993900   39.327900 -14.679100   1.708600  237.662994       1    0.923077
3   27.063101   9.889400  2.3233  0.4371  0.2209  -41.164700   17.103701  -6.429500  59.507099   95.794800       1    0.802395
4   48.629398  15.583500  3.0432  0.3324  0.1861   35.922600   50.127201  -4.310500  71.176697  140.789993       0    0.534653
5   13.123400   9.910200  2.1917  0.8103  0.5370   11.972600    8.239400 -10.211600   9.046000   41.339901       1    0.660000
6   33.078800  13.591000  2.7649  0.3729  0.2088   13.207800   27.735701  -7.145900   2.202500  172.076004     